<a href="https://colab.research.google.com/github/SnehalYu/Query-Auto-Completion-/blob/main/Qac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#LM on QAC data

import zipfile
import requests
import numpy as np

# Download the zip file
url = "https://www.dropbox.com/s/0piap24gagckmba/qac_data.zip?dl=1"
zip_filename = "qac_data.zip"

response = requests.get(url)
with open(zip_filename, 'wb') as f:
    f.write(response.content)

# Extract the zip file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall()

# Specify the file paths
file_path1 = "/content/qac_background.txt"
file_path2 = "/content/qac_training.tsv"
file_path3 = "/content/qac_training.tsv"
file_path4 = "/content/qac_validation.tsv"
# Read the files
with open(file_path1, 'r') as file:
    content1 = file.read()

with open(file_path2, 'r') as file:
    content2 = file.read()

with open(file_path3, 'r') as file:
    content3 = file.read()

with open(file_path4, 'r') as file:
    content4 = file.read()

# Combine the content
trainingset = content1 + content2
trainset = trainingset.lower()  # Convert to lowercase
dataset=trainset+content3.lower()+content4.lower()
# Limit the trainingset size
#trainset = trainset[:50000]

trainset=list(trainset)
trainset.insert(0,'<s>')
trainset.insert(0,'<s>')
trainset.append('</s>')
trainset.append('</s>')

# Create a list of characters
characters = sorted(set(dataset))
characters.append('<s>')
characters.append('</s>')
char_index = {char: index for index, char in enumerate(characters)}

#Count bigrams and trigrams
dict_two = {}
dict_three = {}
trainsetcount = len(trainset)

for i in range(trainsetcount - 1): # -1 modified to -3
    if trainset[i]=='\n':
        pair_two = tuple(['\n','</s>'])
        pair_three = tuple(['\n', '</s>', '</s>'])
    elif trainset[i+1]=='\n':
        pair_two = tuple([trainset[i],'\n'])
        pair_three = tuple([trainset[i],'\n', '</s>'])
    elif i>0 and trainset[i-1] =='\n':
        pair_two = tuple(['<s>','<s>'])
        pair_three = tuple(['<s>','<s>',trainset[i] ])
    elif i>1 and trainset[i-2] =='\n':
        pair_two = tuple(['<s>', trainset[i-1]])
        pair_three = tuple(['<s>',trainset[i-1], trainset[i] ])
    else:
        pair_two = tuple(trainset[i:i + 2])
        pair_three = tuple(trainset[i:i + 3])

    ## pair_two, pair_three declared above
    if pair_two in dict_two:
        dict_two[pair_two] += 1
    else:
        dict_two[pair_two] = 1
    if pair_three in dict_three:
        dict_three[pair_three] += 1
    else:
        dict_three[pair_three] = 1
dict_bi_to_num={value: index for index, value in enumerate(dict_two)}
dict_num_to_bi={index: value for index, value in enumerate(dict_two)}

# Create arrays for bigram and trigram probabilities
num_bigrams = len(dict_two)
num_chars = len(characters)

table = np.zeros((num_bigrams, num_chars))

for i, bigram in enumerate(dict_two):
    for j, char in enumerate(characters):
        trigram = (bigram) + (char,)
        if trigram in dict_three:
            num = dict_three[trigram] + 1
        else:
            num = 1
        denom = dict_two[bigram] + num_chars
        table[i, j] = num / denom # row:bigram, col:trigram



KeyboardInterrupt: ignored

In [ ]:
print(characters)
print(char_index)

['\t', '\n', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<s>', '</s>']
{'\t': 0, '\n': 1, ' ': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38, '<s>': 39, '</s>': 40}


In [ ]:
print(dict_two)
print(dict_three)

{('<s>', '<s>'): 6809416, ('<s>', 'w'): 1112488, ('w', 'w'): 73183, ('w', ' '): 935972, (' ', 's'): 573231, ('s', 'o'): 162798, ('o', 'c'): 94846, ('c', 'i'): 137251, ('i', 'a'): 215033, ('a', 'l'): 504168, ('l', 's'): 148701, ('s', 'e'): 303071, ('e', 'c'): 157217, ('c', 'u'): 69760, ('u', 'r'): 219081, ('r', 'i'): 500109, ('i', 't'): 396041, ('t', 'y'): 207992, ('y', ' '): 522647, (' ', 'g'): 275931, ('g', 'o'): 203168, ('o', 'v'): 108727, ('v', '\n'): 67781, ('\n', '</s>'): 6848733, ('<s>', 'k'): 94083, ('n', 'z'): 5920, ('z', 'i'): 20699, ('i', 'e'): 202399, ('e', ' '): 1088222, (' ', 'd'): 272005, ('d', 'a'): 164761, ('l', 't'): 89264, ('t', 'o'): 378032, ('o', 'n'): 704503, ('n', '\n'): 295807, (' ', 'n'): 228905, ('n', 'y'): 65596, ('y', 'c'): 39926, ('c', ' '): 127098, ('v', ' '): 28811, ('d', 'e'): 348893, ('e', 'f'): 35730, ('f', 'e'): 66386, ('e', 'r'): 1094882, ('r', 'r'): 69721, ('r', 'e'): 616944, ('e', 'd'): 247902, ('d', 'c'): 14266, ('c', 'o'): 1974256, ('o', 'm'): 179

In [ ]:
print(dict_bi_to_num)
print(dict_num_to_bi)

{('<s>', '<s>'): 0, ('<s>', 'w'): 1, ('w', 'w'): 2, ('w', ' '): 3, (' ', 's'): 4, ('s', 'o'): 5, ('o', 'c'): 6, ('c', 'i'): 7, ('i', 'a'): 8, ('a', 'l'): 9, ('l', 's'): 10, ('s', 'e'): 11, ('e', 'c'): 12, ('c', 'u'): 13, ('u', 'r'): 14, ('r', 'i'): 15, ('i', 't'): 16, ('t', 'y'): 17, ('y', ' '): 18, (' ', 'g'): 19, ('g', 'o'): 20, ('o', 'v'): 21, ('v', '\n'): 22, ('\n', '</s>'): 23, ('<s>', 'k'): 24, ('n', 'z'): 25, ('z', 'i'): 26, ('i', 'e'): 27, ('e', ' '): 28, (' ', 'd'): 29, ('d', 'a'): 30, ('l', 't'): 31, ('t', 'o'): 32, ('o', 'n'): 33, ('n', '\n'): 34, (' ', 'n'): 35, ('n', 'y'): 36, ('y', 'c'): 37, ('c', ' '): 38, ('v', ' '): 39, ('d', 'e'): 40, ('e', 'f'): 41, ('f', 'e'): 42, ('e', 'r'): 43, ('r', 'r'): 44, ('r', 'e'): 45, ('e', 'd'): 46, ('d', 'c'): 47, ('c', 'o'): 48, ('o', 'm'): 49, ('m', 'p'): 50, ('p', '\n'): 51, ('<s>', 'm'): 52, ('n', 'h'): 53, ('h', 'u'): 54, ('u', 'n'): 55, ('n', 't'): 56, ('t', ' '): 57, ('n', 'e'): 58, ('e', 't'): 59, ('t', '\n'): 60, ('<s>', 'g'): 6

In [ ]:
print(table)

[[1.46854588e-07 1.46854588e-07 1.46854588e-07 ... 2.85720286e-03
  1.46854588e-07 2.93709175e-07]
 [8.98852974e-07 8.98852974e-07 2.63094265e-03 ... 8.26944736e-05
  8.98852974e-07 8.98852974e-07]
 [7.78433301e-04 9.90112531e-03 5.20717251e-01 ... 1.36567246e-05
  1.36567246e-05 1.36567246e-05]
 ...
 [1.08695652e-02 1.08695652e-02 1.41304348e-01 ... 1.08695652e-02
  1.08695652e-02 1.08695652e-02]
 [1.29870130e-02 1.29870130e-02 1.29870130e-02 ... 1.29870130e-02
  1.29870130e-02 1.29870130e-02]
 [2.08333333e-02 2.08333333e-02 2.08333333e-02 ... 2.08333333e-02
  2.08333333e-02 2.08333333e-02]]


In [ ]:
from typing import List
def get_next_character(context:List, k:int=3):
    assert k>=1
    bigram = tuple(context[-2:])
    if bigram in dict_two:
        probabilities = table[dict_bi_to_num[bigram]]
    else:
        probabilities = 1/table.shape[1] * np.ones(table.shape[1])
        # raise NotImplementedError
        '''
        take any 5 characters, with prob = 1/num_of_characters
        '''
        pass
    top_indices = np.argsort(probabilities)[-k:]
    top_probs = probabilities[top_indices]
    top_probs = np.log(top_probs)
    next_char = [characters[ind] for ind in top_indices]
    next_char.reverse()
    top_probs = list(top_probs)
    top_probs.reverse()
    return next_char, top_probs


def generate_text(starting_context, max_length=10, num_completions=5, topk=3):
    possible_completions = [starting_context]
    probs = [0.0]
    context = starting_context
    # for _ in range(max_length-2):
    for _ in range(max_length-2):
        prob_completions_ = []
        possible_completions_ = []
        for completion,prob in zip(possible_completions, probs):
            ## add termination condition here in if
            if completion[-1] in ['</s>','\n','.'] :
                possible_completions_.extend([completion] )
                prob_completions_.extend([prob])
                # print('end case')
            else:
                next_char, top_probs = get_next_character(completion, k=topk)
                # print('context: ',completion,'next chars: ', str(next_char))
                possible_completions_.extend([completion+[char] for char in next_char] )
                prob_completions_.extend( [prob+prob_ for prob_ in top_probs])
        # select most probables
        prob_completions_ = np.array(prob_completions_)
        max_prob_indices = np.argsort(prob_completions_)[-num_completions:]
        possible_completions = [possible_completions_[ind] for ind in max_prob_indices]
        # print('possible completions:',possible_completions)
        probs = list(prob_completions_[max_prob_indices])
    possible_completions = [''.join(x) for x in possible_completions]
    return possible_completions, probs


userQuery = "gb"
maxLength = 10
starting_context = ['g','b']
generated_text, _ = generate_text(starting_context, maxLength)
print(generated_text)

['gboo comer', 'gboo comes', 'gbrics\n', 'gboo com\n', 'gboo\n']
